In [53]:
import numpy as np
import pandas as pd


def entro(Y=None, pro=None): # calculating entropy
    
    if Y is not None:
        pro = sum(Y == 1) / len(Y)
    if pro == 0 or pro == 1:
        return 0
    return -pro*np.log2(pro) - (1-pro)*np.log2(1-pro)


def midval(X, Y, A): # calculating a value required for information gain value
   
    c = X[:, A]
    return sum(
        (sum(c == a) / len(X)) * entro(Y[c == a])
        for a in set(c)
    )


def finalgain(X, Y, A): # calculating information gain
  
    return entro(Y) - midval(X, Y, A)

 
def gainratio(X, Y, A): # calculating the final gain ratio which is further called in DecisionTreeClassifier
  
    c = X[:, A]
    loop = set(c)
    v = 0
    for i in loop:
        pro = sum(c == i) / len(X)
        if pro == 0 or pro == 1:
            return 0
        v -= pro*np.log2(pro)
    return finalgain(X, Y, A) / v


class DecisionTreeClassifier:
    
    class leaf:
        def __init__(self, p=None):
            self.p = p
            self.A = None
            self.c = {}
            self.posi = None
            self.negi = None

        def __repr__(self):
            return f"({self.posi}+, {self.negi}-)"

    def btree(self, mainr, X, Y, attribute, depth):
        X, Y, attribute = X.copy(), Y.copy(), attribute.copy()
        mainr.posi = sum(Y == 1)
        mainr.negi = len(Y) - mainr.posi
        if mainr.posi == 0 or mainr.negi == 0:
            return
        if depth <= self.m_depth:
            mainr.A = self.bestfeature(X, Y, attribute)
            attribute.add(mainr.A)
            if mainr.A is None:
                return
            for a in set(X[:, mainr.A]):
                mainr.c[a] = self.leaf(p=mainr)
                self.btree(mainr.c[a],
                                 X[X[:, mainr.A] == a],
                                 Y[X[:, mainr.A] == a],
                                 attribute,
                                 depth+1)
    def error(self, X, Y):
        e = np.array([self.prediction(x) for x in X])
        return print((sum(e != Y) / len(Y)))

    def accuracy(self, X, Y):
        e = np.array([self.prediction(x) for x in X])
        return print((sum(e == Y) / len(Y))*100)

    
    def prediction(self, x):
        assert hasattr(self, 'mainr'), "not trained yet"
        n = self.mainr
        while n.c:
            try:
                n = n.c[x[n.A]]
            except KeyError:
                break
        return int(n.posi > n.negi)

    def show_tree(self):
        self.treeprint(self.mainr, depth=0)


    def fitter(self, X, Y, m_depth=None):
        self.m_depth = X.shape[1] if m_depth is None else m_depth
        self.mainr = self.leaf()
        self.btree(self.mainr, X, Y, set(), depth=1)

    def bestfeature(self, X, Y, zat):
        best_gain, best_feature = 0, None
        for A in range(X.shape[1]):
            if A in zat:
                continue
            gain = gainratio(X, Y, A)
            if gain > best_gain:
                best_gain = gain
                best_feature = A
        return best_feature

    
    def treeprint(self, mainr, depth):
        print(mainr)
        for k, v in mainr.c.items():
            print("\t"*(depth+1) + f"Node:[x{mainr.A+1} = {k}] ", end='')
            self.treeprint(v, depth+1)


In [65]:
Decisiontrain = pd.read_csv('tic-tac-toe_train.csv',header=None)
Xtrain = np.asarray(Decisiontrain)[:,:9]
Ytrain = (np.asarray(Decisiontrain)[:,-1] == 'win').astype(int)
Decisiontest =pd.read_csv('tic-tac-toe_test.csv',header=None)
Xtest = np.asarray(Decisiontest)[:,:9]
Ytest = (np.asarray(Decisiontest)[:,-1] == 'win').astype(int)

obj = DecisionTreeClassifier()
obj.fitter(Xtrain, Ytrain, m_depth=9)
print("Accuracy for training:")
obj.accuracy(Xtrain,Ytrain)
print("Accuracy for test:")
obj.accuracy(Xtest, Ytest)

Accuracy for training:
100.0
Accuracy for test:
73.0


In [56]:
obj.fitter(Xtrain, Ytrain, m_depth=6)
print("Error for training:")
obj.error(Xtrain,Ytrain)
print("Error for test:")
obj.error(Xtest, Ytest)

Error for training:
0.0
Error for test:
0.27


In [66]:
obj.show_tree()

(165+, 85-)
	Node:[x5 = x] (93+, 20-)
		Node:[x1 = o] (29+, 13-)
			Node:[x3 = x] (13+, 5-)
				Node:[x7 = o] (3+, 5-)
					Node:[x4 = x] (2+, 0-)
					Node:[x4 = b] (1+, 0-)
					Node:[x4 = o] (0+, 5-)
				Node:[x7 = b] (1+, 0-)
				Node:[x7 = x] (9+, 0-)
			Node:[x3 = b] (9+, 0-)
			Node:[x3 = o] (7+, 8-)
				Node:[x2 = x] (5+, 1-)
					Node:[x8 = b] (0+, 1-)
					Node:[x8 = x] (5+, 0-)
				Node:[x2 = b] (2+, 0-)
				Node:[x2 = o] (0+, 7-)
		Node:[x1 = b] (26+, 5-)
			Node:[x7 = o] (9+, 4-)
				Node:[x8 = o] (2+, 4-)
					Node:[x9 = o] (0+, 4-)
					Node:[x9 = b] (1+, 0-)
					Node:[x9 = x] (1+, 0-)
				Node:[x8 = b] (2+, 0-)
				Node:[x8 = x] (5+, 0-)
			Node:[x7 = b] (2+, 1-)
				Node:[x3 = o] (0+, 1-)
				Node:[x3 = x] (2+, 0-)
			Node:[x7 = x] (15+, 0-)
		Node:[x1 = x] (38+, 2-)
			Node:[x9 = o] (6+, 2-)
				Node:[x4 = x] (3+, 0-)
				Node:[x4 = b] (1+, 0-)
				Node:[x4 = o] (2+, 2-)
					Node:[x3 = x] (2+, 1-)
						Node:[x6 = x] (0+, 1-)
						Node:[x6 = b] (1+, 0-)
						Node: